In [1]:
# project: p1
# submitter: wlangas
# partner: none
# hours: 15

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
from pandas import Series, DataFrame
from subprocess import check_output
import json, ast, os, random, time

In [3]:
check_output("pwd", shell=True)

b'/home/wlangas/p1\n'

In [6]:
check_output(["git", "checkout", "main"], cwd = "repo")

CalledProcessError: Command '['git', 'checkout', 'main']' returned non-zero exit status 128.

In [ ]:
#q1
git_log = str(check_output(["git", "log"], cwd = "repo"), encoding="utf-8")
commits = git_log.split()
commit_count = 0

for word in commits:
    if word == "commit":
        commit_count += 1
        
commit_count

In [ ]:
#q2 - Git log, count number of commits by each author
author_commits = {"Steve": 0, "Ada": 0, "Linus": 0}

for word in commits:
    try:
        author_commits[word] += 1
    except:
        pass
    
author_commits

In [ ]:
#q3
author_series = Series(author_commits)

ax = author_series.plot.bar(color="k", figsize=[3,3], ylabel="Commit Count", fontsize=17)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_yticks([0,1,2,3])
ax.yaxis.label.set_size(17)

In [ ]:
#q4
# https://stackoverflow.com/questions/3137094/how-to-count-lines-in-a-document
commit_ids = list()
for word in commits: #Pull out the 40 digit commit IDs
    if len(word) == 40:
        commit_ids.append(word)

line_count = list()
for git_id in commit_ids:
    check_output(["git", "checkout", git_id], cwd="repo")
    line_count.append(str(check_output(["wc", "-l", "wc.py"], cwd="repo")))
    check_output(["git", "checkout", "main"], cwd = "repo")
                      
line_count
for idx in range(len(line_count)):
    line_count[idx] = int(line_count[idx][2:4])
    
line_count = line_count[::-1]
line_count

In [ ]:
#q5 - Graph previous list
ser = pd.Series(line_count)

ax2 = ser.plot.line(ylabel="Lines", xlabel="Commit")
ax2.set_yticks([0,10,20,30,40])
ax2.set_xticks([0,2,4,6])
ax2.spines["top"].set_visible(False)
ax2.spines["right"].set_visible(False)

In [ ]:
# test inputs to exercise wc.py

# expected result: {'X': 1, 'Y': 1, 'Z': 1}
test1 = """X Y Z"""

# expected result: 
test2 = """A B
A C
"""

# expected result: {'A': 2, 'B': 1, 'C': 1}
test3 = """A B
a c
"""

In [ ]:
#q6
def run_wc(body, commit=None):
    
    f = open('test.txt', 'w')
    f.write(body)
    f.close()

    # deal with the commit
    if (commit == None):
        check_output(["git", "checkout", "main"], cwd="repo")
    else:
        check_output(["git", "checkout", commit], cwd="repo")
    
    try:
        test_txt = (check_output("pwd", shell=True)).decode("utf-8").replace("\n", "")
        test_txt = os.path.join(test_txt, "repo")
        
        output = check_output(["python3", "wc.py", os.path.join("..","test.txt"), "ALL"], cwd="repo")
        output = output.decode("utf-8").replace("\n", "").replace("  "," ").replace("\\","")
        return json.loads(output)
    except:      
        return None
                
test1_result = run_wc("X Y Z", None)
test1_result

In [ ]:
#q7
test3_result = run_wc("A B a c", None)
test3_result

In [ ]:
#q8
author_log = list()
for word in commits:
    if word in ["Steve", "Ada", "Linus"]:
        author_log.append(word)
author_log

def test_table(body, expected):

    test_passes = list()
    
    test_txt = (check_output("pwd", shell=True)).decode("utf-8").replace("\n", "")
    test_txt = os.path.join(test_txt, "repo")
        
    for commit in commit_ids:
        try:
            output = run_wc(body, commit)
            
            if (str(output) == str(expected)):
                test_passes.append(True)
            else:
                test_passes.append(False)
        except Exception as err: 
            #print(err)
            test_passes.append(False)
            
    formatted_data = {"commit": commit_ids[::-1], "author": author_log[::-1], "pass": test_passes[::-1]}
    result = pd.DataFrame(formatted_data)
    
    return result

table1_result = test_table(test1, {'X': 1, 'Y': 1, 'Z': 1})
table1_result

In [ ]:
#q9
table2 = test_table(test2, {'A': 2, 'B': 1, 'C': 1})
table2

In [ ]:
#q10
table3 = test_table(test3, {'A': 2, 'B': 1, 'C': 1})
table3

In [ ]:
#I would assume the 3rd test is best for testing. This is because it has a greater range of inputs because
# of the lowercase letters, which leads to one more instance being deemeds as a failure than test2, which is commit
# 761627a3ad9ee681f27211326e9149f01764ceee by Linus

In [ ]:
'''
description: randomly generate input for wc.py and time how long it takes to run
uniq_words: the function should randomly generate this many words and put them in a list
total_words: the function should randomly sample this many words from the above list to produce an input string
word_size: how many characters long each word should be
commit: what version of wc.py to run on the random input
return value: how many seconds it took to run wc.py (not counting the time to generate the input file)
Hint: Try taking a look at the random module's choice function...   
'''
def time_run_sec(uniq_words, total_words, word_size=6, commit=None):
    
    #Generate uniq_words number of unique words and put in a list
    letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    uniq_list = list()
    for idx in range(uniq_words):
        curr_word = ""
        for count in range(word_size):
            curr_word += random.choice(letters)
        uniq_list.append(curr_word)
    
    #Sample total_words number of words from the above list to produce an input string
    input_string = ""
    for idx in range(total_words):
        input_string += random.choice(uniq_list) + " "
        
        
    start_time = time.time()
    run_wc(input_string, commit)
    end_time = time.time()
    
    return end_time - start_time

In [ ]:
versions = {
    "v0-baseline": "6f5ca9327e986315ffcacddce5d9d6195c0913b7",
    "v1-open-once": "f37e610ce055a3d894baac2d9449e6eb77c72320",
    "v2-pass-per-uniq": "c10b5a6cb4f06c96f6f221df2d5ec33af767d5c5",
    "v3-single-pass": "4e4128313b8d5b5e5d04f2e8e585f64f7c5831a4",
}

In [ ]:
#q11
versions_list = list(versions.keys())
exec_times = list()
for version in versions_list:
    exec_times.append(1000 * time_run_sec(100, 5000, commit=versions[version]))

formatted_data = {"Version": versions_list, "Exec Time (ms)": exec_times}
formatted_df = DataFrame(formatted_data)

ax2 = formatted_df.plot.barh(color='k', legend=False, figsize=[3,2])
ax2.set_xlabel("Exec Time (ms)")
ax2.set_yticklabels(versions_list)
ax2.set_xscale('log')
ax2.invert_yaxis()
ax2.set_xticks([10,100,1000])
ax2.spines["top"].set_visible(False)
ax2.spines["right"].set_visible(False)

In [ ]:
#q12
versions_list = list(versions.keys())
exec_times = list()
for version in versions_list:
    exec_times.append(1000 * time_run_sec(1, 5000, commit=versions[version]))

formatted_data = {"Version": versions_list, "Exec Time (ms)": exec_times}
formatted_df = DataFrame(formatted_data)

ax2 = formatted_df.plot.barh(color='k', legend=False, figsize=[3,2])
ax2.set_xlabel("Exec Time (ms)")
ax2.set_yticklabels(versions_list)
ax2.set_xscale('log')
ax2.invert_yaxis()
ax2.set_xticks([10,100,1000])
ax2.spines["top"].set_visible(False)
ax2.spines["right"].set_visible(False)

In [ ]:
#q13

list_percents = [10, 5, 1]
list_total_words = [1000, 2000, 5000, 10000]

def total_to_unique(version):
    list_10 = list()
    for num in list_total_words:
        list_10.append(time_run_sec(int((0.1 * num)), num, commit=versions[version]))

    list_5 = list()
    for num in list_total_words:
        list_5.append(time_run_sec(int((0.05 * num)), num, commit=versions[version]))

    list_1 = list()
    for num in list_total_words:
        list_1.append(time_run_sec(int((0.01 * num)), num, commit=versions[version]))

    formatted_data = {"Total Words": [1000, 2000, 5000, 10000], "10": list_10, "5": list_5, "1": list_1}
    return pd.DataFrame(formatted_data)

df_2 = total_to_unique("v2-pass-per-uniq")
fig, ax = plt.subplots(ncols=2, nrows=1, sharey=True, figsize=[10,4])

df_2.plot.line(ax=ax[0],x="Total Words", legend=False)
ax[0].set_title("version 2")
ax[0].set_ylabel("Time (s)")
ax[0].set_xticks([0,2000,4000,6000,8000,10000])
ax[0].set_yticks([0.1,0.2,0.3,0.4])

df_3 = total_to_unique("v3-single-pass")
df_3.plot.line(ax=ax[1], x="Total Words", legend=True)
ax[1].set_title("version 3")

In [ ]:
"""
If repo is a Repo instance, calling repo.commit(...) should create a new Commit object, 
add the new Commit to some structures in the Repo (list and/or dict), and return the Commit.

str(commit) on a Commit object should give something like 'A39DE2[Aarushi]: created the repo'.

repo[????] should produce a Commit object. ???? could be an integer index, in which case 0 would be the first commit,
1 would be the second, and -1 would be the last, etc.

If ???? is a string, repo[????] should produce the Commit"""
class Commit:
    def __init__(self, commit_hex, author, message):
        self.commit_hex = commit_hex
        self.author = author
        self.message = message
        
    def __str__(self):
        return str(self.commit_hex + "[" + self.author + "]: " + self.message)

In [ ]:
class Repo:
    def __init__(self):
        self.commits = list()
        self.commits_dict = dict()
    
    def commit(self, commit_hex, author, message):
        new_commit = Commit(commit_hex, author, message)
        self.commits.append(new_commit)
        self.commits_dict[commit_hex] = new_commit
        return new_commit
    
    def __getitem__(self, key):
        if isinstance(key, int):
            return self.commits[key]
        else:
            key = key.strip()
            return self.commits_dict[key.upper()]

In [ ]:
#q14
repo = Repo()
c1 = repo.commit("A39DE2", "Aarushi", "created the repo")
c2 = repo.commit("D1E9C5", "Ben", "wrote some code")
c3 = repo.commit("AA0912", "Emma", "added a readme")
c4 = repo.commit("B143D0", "Hunter", "made some tests")
c5 = repo.commit("F0148D", "Jaime", "fixed a bug")
c6 = repo.commit("B978EE", "Wendy", "optimized the code")
c1.commit_hex, c2.author, c3.message, c4.commit_hex, c5.author, c6.message

In [ ]:
#q15
str(c1), str(repo[1]), str(repo[-1])

In [ ]:
#q16
str(repo["F0148D"]), str(repo["b143d0"]), str(repo["  AA0912  "])